In [35]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
# Télécharger les données de stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score



import matplotlib.pyplot as plt

# Charger les DataFrames)
df = pd.read_csv("Data_Projet/Datafiniti_Hotel_Reviews.csv", sep=',', usecols=range(20))

# Remplacer les valeurs NaN dans la colonne "reviews.title" par une chaîne vide
df["reviews.title"] = df["reviews.title"].fillna("")

# Sélectionner les colonnes qui nous intéressent
df = df[["name", "reviews.rating", "reviews.text", "reviews.title"]]

# Concaténer les colonnes reviews.text et reviews.title
df["reviews_combined"] = df["reviews.title"] + " " + df["reviews.text"]

# Trier le DataFrame par la colonne "reviews.rating"
df = df.sort_values(by="reviews.rating")

# Supprimer les colonnes "reviews.text" et "reviews.title"
df.drop(columns=["reviews.text", "reviews.title"], inplace=True)

df["reviews_combined"] = df["reviews_combined"].fillna("")
stop_words = set(stopwords.words("english"))
# Initialiser le lemmatiseur
lemmatizer = WordNetLemmatizer()

# Fonction pour retirer les stopwords d'une chaîne de texte
def remove_stopwords(text):
    # Tokeniser le texte en mots
    tokens = word_tokenize(text)
    # Retirer les mots qui sont des stopwords
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    # Lemmatisation des mots
    lemmatized_tokens = [lemmatizer.lemmatize(word.lower()) for word in filtered_tokens]
    # Rejoindre les tokens lemmatisés en une chaîne de texte
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text

# Appliquer la fonction remove_stopwords à la colonne "reviews_combined"
df["reviews_combined_no_stopwords"] = df["reviews_combined"].apply(remove_stopwords)

tfidf_vectorizer = TfidfVectorizer(
    lowercase=True,  # Convertit le texte en minuscules
    tokenizer=word_tokenize,  # Tokenise les mots
    stop_words='english',  # Retire les mots vides en anglais
    max_df=0.8,  # Ignore les mots qui apparaissent dans plus de 80% des documents
    min_df=5  # Ignore les mots qui n'apparaissent pas au moins dans 5 documents
)

tfidf_matrix = tfidf_vectorizer.fit_transform(df["reviews_combined_no_stopwords"])

# Sélectionner l'indice d'un document particulier pour visualiser ses valeurs TF-IDF
# index_document = 0

# Extraire les valeurs TF-IDF pour le document sélectionné
# tfidf_values = tfidf_matrix[index_document].toarray()[0]

# Obtenir les termes associés à chaque dimension de la matrice TF-IDF
# terms = tfidf_vectorizer.get_feature_names_out()

# Créer un DataFrame pour stocker les termes et leurs valeurs TF-IDF correspondantes
# tfidf_df = pd.DataFrame({'Term': terms, 'TF-IDF Value': tfidf_values})

# Trier le DataFrame par les valeurs TF-IDF dans l'ordre décroissant
# tfidf_df_sorted = tfidf_df.sort_values(by='TF-IDF Value', ascending=False)

# Sélectionner les 10 termes les plus importants avec les valeurs TF-IDF correspondantes
# top_terms = tfidf_df_sorted.head(10)

# Créer un graphique à barres pour visualiser les 10 termes les plus importants
# plt.figure(figsize=(10, 6))
# plt.barh(top_terms['Term'], top_terms['TF-IDF Value'])
# plt.xlabel('TF-IDF Value')
# plt.ylabel('Term')
# plt.title('Top 10 TF-IDF Values for Document 1')
# plt.gca().invert_yaxis()
# plt.show()

# Obtenir la taille de la matrice TF-IDF
# nombre_documents, nombre_termes = tfidf_matrix.shape

# Afficher la taille de la matrice
# print("Taille de la matrice TF-IDF : {} lignes (documents) x {} colonnes (termes)".format(nombre_documents, nombre_termes))



# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df["reviews.rating"], test_size=0.2, random_state=42)

# Convertir la matrice TF-IDF au format numpy array
X_train_np = X_train.toarray()
y_train_np = np.array(y_train)

# Division des données en ensembles d'entraînement et de validation
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train_np, y_train_np, test_size=0.2, random_state=42)

# Afficher les tailles des ensembles d'entraînement et de test
print("Taille de l'ensemble d'entraînement :", X_train.shape[0])
print("Taille de l'ensemble de test :", X_test.shape[0])


# Construction du modèle
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(len(tfidf_vectorizer.vocabulary_),)),  # Couche d'entrée
    tf.keras.layers.Dense(64, activation='relu'),  # Couche cachée
    tf.keras.layers.Dense(1, activation='sigmoid')  # Couche de sortie
])

# Compilation du modèle
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Entraînement du modèle en utilisant les données d'entraînement et de validation
model.fit(X_train_split, y_train_split, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Évaluation du modèle sur l'ensemble de test
loss, accuracy = model.evaluate(X_test, y_test)
print("Loss :", loss)
print("Accuracy :", accuracy)




[nltk_data] Downloading package punkt to /Users/lucas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/lucas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/lucas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/Users/lucas/Documents/A5/4-VM IA/cours2/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Taille de l'ensemble d'entraînement : 8000
Taille de l'ensemble de test : 2000
Epoch 1/10
200/200 [==============================] - 1s 2ms/step - loss: -383.8014 - accuracy: 0.0564 - val_loss: -1679.7439 - val_accuracy: 0.0531
Epoch 2/10
200/200 [==============================] - 0s 2ms/step - loss: -7223.8174 - accuracy: 0.0573 - val_loss: -16312.2041 - val_accuracy: 0.0531
Epoch 3/10
200/200 [==============================] - 0s 2ms/step - loss: -34066.4492 - accuracy: 0.0573 - val_loss: -57584.4961 - val_accuracy: 0.0531
Epoch 4/10
200/200 [==============================] - 0s 2ms/step - loss: -93098.1484 - accuracy: 0.0573 - val_loss: -136318.4844 - val_accuracy: 0.0531
Epoch 5/10
200/200 [==============================] - 0s 2ms/step - loss: -193991.0469 - accuracy: 0.0573 - val_loss: -261062.5000 - val_accuracy: 0.0531
Epoch 6/10
200/200 [==============================] - 0s 2ms/step - loss: -344488.0312 - accuracy: 0.0573 - val_loss: -438911.7188 - val_accuracy: 0.0531
Epoch 7/

2024-03-03 21:23:14.160888: W tensorflow/core/framework/op_kernel.cc:1827] INVALID_ARGUMENT: TypeError: 'SparseTensor' object is not subscriptable
Traceback (most recent call last):

  File "/Users/lucas/Documents/A5/4-VM IA/cours2/venv/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    return func(device, token, args)

  File "/Users/lucas/Documents/A5/4-VM IA/cours2/venv/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 146, in __call__
    outputs = self._call(device, args)

  File "/Users/lucas/Documents/A5/4-VM IA/cours2/venv/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 153, in _call
    ret = self._func(*args)

  File "/Users/lucas/Documents/A5/4-VM IA/cours2/venv/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Users/lucas/Documents/A5/4-VM IA/cours2/venv/lib/python3.10/site-packages/keras/src/engine/data

InvalidArgumentError: Graph execution error:

Detected at node EagerPyFunc defined at (most recent call last):
<stack traces unavailable>
TypeError: 'SparseTensor' object is not subscriptable
Traceback (most recent call last):

  File "/Users/lucas/Documents/A5/4-VM IA/cours2/venv/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    return func(device, token, args)

  File "/Users/lucas/Documents/A5/4-VM IA/cours2/venv/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 146, in __call__
    outputs = self._call(device, args)

  File "/Users/lucas/Documents/A5/4-VM IA/cours2/venv/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 153, in _call
    ret = self._func(*args)

  File "/Users/lucas/Documents/A5/4-VM IA/cours2/venv/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Users/lucas/Documents/A5/4-VM IA/cours2/venv/lib/p